# CME538 - Introduction to Data Science
## Assignment 5 - Geospatial Analysis

### Learning Objectives
After completing this assignment, you should be comfortable:
- Visualization of Geospatial Data.
- Using Geosptial Python packages Geopandas and Folium.
- Working with Coordinate Reference Systems.
- Spatia

Note: You are free to add new cells to use as a scratch pad, but make sure to clean you code up and present your answer in the cell indicated with `# Write your code here`.

### Marking Breakdown

Question | Points
--- | ---
Question 1a | 1
Question 1b | 1
Question 2 | 1
Question 3 | 1
Question 4 | 1
Question 5a | 1
Question 5b | 1
Question 5c | 1
Question 6 | 1
Question 7 | 1
Question 8a | 1
Question 8b | 1
Question 8c | 1
Question 8d | 1
Total | 14

One of the following marks below will be added to the **Total** above.

### Code Quality

| Rank | Points | Description |
| :-- | :-- | :-- |
| Youngling | 1 | Code is unorganized, variables names are not descriptive, redundant, memory-intensive, computationally-intensive, uncommented, error-prone, difficult to understand. |
| Padawan | 2 | Code is organized, variables names are descriptive, satisfactory utilization of memory and computational resources, satisfactory commenting, readable. |
| Jedi | 3 | Code is organized, easy to understand, efficient, clean, a pleasure to read. #cleancode |

## Setup Notebook

In [ ]:
# Import 3rd party libraries
import os
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# Overview
This assignment continues on from Assignment 4. Remember, You've just been hired by the City of Toronto. Congratulation! In Assignment 4, you explored patterns in user behavior. In this assignment, you'll continue your exploration using geospaital visualization and data manipulation tools.


# 1. Get Station Data
First, let's get the station information from `https://tor.publicbikesystem.net`. Remember, we learned about `requests` in Week 3.

In [ ]:
response = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')
bikeshare_stations = pd.DataFrame(json.loads(response.content)['data']['stations'])[['station_id', 'name', 'lat', 'lon', 'capacity']].astype({'station_id': 'int',})
bikeshare_stations = bikeshare_stations.rename(columns={'station_id': 'Station Id', 
                                                        'name': 'Station Name'})

# View DataFrame
bikeshare_stations.head()

# 2. Import Toronto Neighbourhoods shapefile
## Question 1a
The shapefile for City of Toronto neighbourhood boundaries `'toronto_neighbourhoods.shp'` is included in the assignment directory. Use `gpd.read_file()` to open it and save its contents to a new variables called `neighbourhoods`. Make note of the `'geometry'` column. This is what makes a DataFrame a Geopandas GeoDataFrame.

In [ ]:
# Write your code here.
neighbourhoods = gpd.read_file('toronto_neighbourhoods.shp')

# View GeoDataFrame
neighbourhoods.head()

## Question 1b
`neighbourhoods` contains a number of columns we don't need and with generic names. Drop all columns except for `'geometry', and 'FIELD_8'`. Next, change the name of column `'FIELD_8'` to `'name'`. Lastly, remove the neighbourhood id from the name. For example, `'Yorkdale-Glen Park (31)'` should become `'Yorkdale-Glen Park'`.  

In [ ]:
# Write your code here.
neighbourhoods = neighbourhoods.filter(['geometry','FIELD_8'])

#rename columns
neighbourhoods = neighbourhoods.rename(columns={'FIELD_8': 'name'})

#remove digits from 'name' column
neighbourhoods['name'] = neighbourhoods['name'].str.replace(r'\d+', '')

#remove '()' from 'name' column
neighbourhoods['name'] = neighbourhoods['name'].str.strip('()')

# View GeoDataFrame
neighbourhoods.head()

The `'geometry'` column can contain a variety of different shapes as seen below, however, the most common are `Point, LineString, and Polygon`. 
<br>
<img src="images/shapes.png" alt="drawing" width="450"/>
<br> 
In the case of our Toronto neighbourhoods dataset, lets take a look at the contents of the `'geometry'` column.

In [ ]:
# View the first few entries in the "geometry" column
neighbourhoods.geometry.head()

Neighbourhood geometries are saved as `POLYGON` shapes.

# 3. Import Toronto Bikeway Network
The city of Toronto publishes data about its current Bikeway Network. Let's import `'bikeway_network.shp'` and take a look.

In [ ]:
bike_lanes = gpd.read_file('bikeway_network.shp')
bike_lanes.head()

Let's take a look at the `'geometry'` column.

In [ ]:
bike_lanes.geometry.head()

You'll see that the bike lane geometries are saved as `LINESTRING` shapes.

Next, let's clean this GeoDataFrame up. First, we'll grab only the columns we're interested in and rename them.

In [ ]:
bike_lanes = bike_lanes[['LF_NAME', 'SEG_TYPE', 'length', 'geometry']]
bike_lanes = bike_lanes.rename(columns={'LF_NAME': 'name', 'SEG_TYPE': 'route_type'})
bike_lanes.head()

We can see that there are many different types of bike routes in this dataset.

In [ ]:
bike_lanes['route_type'].unique()

For our analysis, we're just interested in bike lanes so let's filter out all other route types.

In [ ]:
bike_lanes = bike_lanes[bike_lanes['route_type'] == 'bike lane']
bike_lanes.head()

# 4. Convert Station Locations DataFrame to a GeoDataFrame
You'll remember that we imported the station location data as a stantard Pandas DataFrame.

In [ ]:
bikeshare_stations.head()

Therefore, we'll need to convert it to a GeoDataFrame to have the special `'geometry'` column like `neighbourhoods` and `bike_lanes`.

## Question 2
Use `gpd.GeoDataFrame()` to create a GeoDataFrame from `bikeshare_stations` and assign the new GeoDataFrame to a new variable called `bikeshare_stations_gdf`.

Hint: The **geometry** argument for `gpd.GeoDataFrame()` can be assigned using [`gpd.points_from_xy()`](https://geopandas.org/reference/geopandas.points_from_xy.html) where (x,y) is (lon,lat).

In [ ]:
# Write your code here.
bikeshare_stations_gdf = gpd.GeoDataFrame(bikeshare_stations, 
                                       geometry=gpd.points_from_xy(bikeshare_stations.lon, 
                                                                   bikeshare_stations.lat))

# View DataFrame
bikeshare_stations_gdf.head()

You should see that `bikeshare_stations_gdf` has a new column `'geometry'` with `POINT` shapes.

In [ ]:
bikeshare_stations_gdf.geometry.head()

# 5. Generate Simple Plot
We can quickly visualize the data with the plot() method. Let's try it for `neighbourhoods`.

In [ ]:
neighbourhoods.plot(figsize=(15, 8), edgecolor='w', alpha=0.75);

Looks great! Let's now try layering all three datasets on top of each other.

In [ ]:
ax = neighbourhoods.plot(figsize=(15, 8), edgecolor='w', alpha=0.75)
bikeshare_stations_gdf.plot(ax=ax, color='orange', edgecolor='k', label='Bike Stations')
bike_lanes.plot(ax=ax, color='green', label='Bike Lanes');

Hmmmmmmm, something funny is going on here. We can see a cluster of orange points close to (0, 0) and a green cluster near (-9e6, 6e6). This suggests that we have different coordinate systems for our geometries.

In [ ]:
neighbourhoods.geometry.head(1)

In [ ]:
bikeshare_stations_gdf.geometry.head(1)

In [ ]:
bike_lanes.geometry.head(1)

We can see that `neighbourhoods` and `bikeshare_stations_gdf` is in latitude and longitude, while `bike_lanes` is in some other coordinate system.

# 6. Coordinate Reference System (CRS)
A coordinate reference system (CRS) shows how projected points correspond to real locations on Earth. As you can see from the [figure](https://www.reddit.com/r/MapPorn/comments/b5yaf5/an_incomplete_list_of_map_projections/) below, there are quite a few different Coordinate Reference Systems. 
<br>
<img src="images/crs.png" alt="drawing" width="700"/>
<br> 
When we create a GeoDataFrame from a shapefile, the CRS is already imported. We can check out the `crs` for each GeoDataFrame using the `.crs` method. Let's check it out for each of our GeoDataFrames.

In [ ]:
neighbourhoods.crs

Looks like the `crs` for `neighbourhoods` is `EPSG:4326`

In [ ]:
bikeshare_stations_gdf.crs

The `bikeshare_stations` GeoDataFrame does not contain `crs` information because we contructed it ourselves from (lat,lon) coordinates. However, we know from `publicbikesystem.net` that the station locations have the same `crs` as `neighbourhoods`.

So, let's set the crs of `bikeshare_stations_gdf` (`EPSG:4326`).

In [ ]:
bikeshare_stations_gdf.crs = {'init': 'epsg:4326'}

Now, let's check.

In [ ]:
bikeshare_stations_gdf.crs

It worked!

In [ ]:
bike_lanes.crs

`bike_lanes` shows a different `crs` from `neighbourhoods` and `bikeshare_stations_gdf` (`EPSG:3857`).

EPSG stands for [European Petroleum Survey Group](https://epsg.org/home.html|).

### EPSG:4326 (WGS84)
The World Geodetic System of 1984 is the geographic coordinate system (the three-dimensional one) used by GPS to express locations on the earth. WGS84 is the defined coordinate system for GeoJSON, as longitude and latitude in decimal degrees. For the most part, when you describe a lon/lat coordinate location, it’s based on the EPSG:4326 coordinate system. 

### EPSG:3857 (Pseudo-Mercator)
The projected Pseudo-Mercator coordinate system takes the WGS84 coordinate system and projects it onto a square. EPSG:3857 was adopted by Google Maps in 2005.

You can take an entire course on geographic information systems (GIS). For the purpose of this short introduction to geospatial data analysis in Python, the important take-aways are:
1. There are many different `crs`'s out there and you should be aware that different shapefiles may use different `crs`'s.
2. When analyzing and plotting multiple GeoDataFrames, it's important that they all use the same CRS. 

So, let's transform `bike_lanes` from `EPSG:3857` to `EPSG:4326`. 

In [ ]:
bike_lanes = bike_lanes.to_crs(epsg=4326)
bike_lanes.crs

Remember, the `crs` you use depends on the type of analysis or visualization you're undertaking. For example:
- Equal-area projections preserve area. This is a good choice, if you'd like to calculate the area of a country, province, city, or neighbourhood.
- Equidistant projections preserve distance. This would be a good choice for calculating distance of a shipping route.

Map projections are not 100% accurate and each projection distorts the surface of the Earth in one way or another while retaining some useful properties specific to its intended use. 

## Question 3
Because we're working in a relatively small area (Toronto) and we'll be looking at areas and distances, we should transform to a 2D projection `crs`. The City of Mississauga distributes datasets in `UTM NAD83 Zone 17N (EPSG:26917)` so let's try that. Transform `neighbourhoods, bikeshare_stations_gdf, and bike_lanes` to `EPSG:26917`.

In [ ]:
# Write your code here.
bike_lanes = bike_lanes.to_crs(epsg=26917)
bikeshare_stations_gdf = bikeshare_stations_gdf.to_crs(epsg=26917)
neighbourhoods = neighbourhoods.to_crs(epsg=26917)

In [ ]:
bike_lanes.crs

## Question 4
Now, let's try plotting all three GeoDataFrame together again. Don't forget to include the following axis labels:
```python
plt.xlabel('East, meters', fontsize=18)
plt.ylabel('North, meters', fontsize=18)
```
and a legend showing the following labels:
- Bike Stations
- Bike Lanes

Your figure should look something like this.
<br>
<img src="images/q4.png" alt="drawing" width="500"/>
<br> 

In [ ]:
# Write your code here.

# Plot bike Lane shape file 
ax = bike_lanes.plot(figsize=(15, 8), edgecolor='purple', label='Bike Lanes', alpha = 0.75)

# Plot the neighbourhoods location
neighbourhoods.plot(figsize=(15, 8), ax=ax, edgecolor='white')

# Plot the bikeshare station location
bikeshare_stations_gdf.plot(ax=ax ,figsize=(15, 8), color='orange', edgecolor = 'k' , label='Bike Stations')


# plot legend, axis and display
plt.legend(fontsize=16, loc=2)
plt.xlabel('East, meters', fontsize=18)
plt.ylabel('North, meters', fontsize=18)
plt.show()


# 7. Geometric Object Attributes
As you'll recall, the `'geometry'` column of a GeoDataFrame can contain many different types of geometies including:
- POLYGON
- LINESTRING
- POINT

These types of geometries have built in attributes that we can use to quickly analyze the dataset.

### POINT: X and Y Coordinates

In [ ]:
bikeshare_stations_gdf.geometry.x.head()

In [ ]:
bikeshare_stations_gdf.geometry.y.head()

### LINESTRING: Distance (meters)

In [ ]:
bike_lanes.geometry.length.head()

### POLYGON: Area (meters$^2$)

In [ ]:
neighbourhoods.geometry.area.head()

# 8. Neighbourhoods Analysis
The City of Toronto has expanded its bike share infrastructure over the past few years and has tried to do it in a way that provides access as equitably as possible while providing relief on high-flow transit routes. Before further expanding the the city's bike share network, the city would like to analyze the density of bike stations for each neighbourhood in Toronto.

The figure below is hard to interpret because some areas have many bike stations which results in overplotting at this scale.

In [ ]:
ax = neighbourhoods.plot(figsize=(15, 8), edgecolor='w', alpha=0.75)
bikeshare_stations_gdf.plot(ax=ax, color='orange', edgecolor='k', label='Bike Stations')
plt.legend(fontsize=16)
plt.xlabel('East, meters', fontsize=18)
plt.ylabel('North, meters', fontsize=18);

## Question 5a
Calculate the area in square kilometers of each neighbourhood and save this information as a new column named `'area'` in the `neighbourhoods` GeoDataFrame.

In [ ]:
# Write your code here.
neighbourhoods['area'] = neighbourhoods['geometry'].area
# View GeoDataFrame
neighbourhoods.head()

## Question 5b
Calculate the number of bike stations in each neighbourhood and save this information as a new column named `'stations'` in the `neighbourhoods` GeoDataFrame. Your GeoDataFrame should look like this.
<br>
<img src="images/station_count.png" alt="drawing" width="500"/>
<br> 
Lastly, sort `neighbourhoods` by the number of stations (`'stations'`) in descending oder.

Hints:
- You can use `.apply()` to loops through each neighbourhood. 
```python
neighbourhoods.apply(lambda row: your_calculation, axis=1)
```
- You can use the `.within()` GeoDataFrame method to check if a bike station is in a neighbourhood.
- After you're finished the calculation, sort `neighbourhoods` by the `'stations'` column in descending oder.

In [ ]:
# Write your code here.
neighbourhoods['stations'] = neighbourhoods.apply(lambda row: bikeshare_stations_gdf.within(row.geometry).sum(), axis=1)

#sort values based on 'stations' column
neighbourhoods = neighbourhoods.sort_values('stations', ascending = False)
# View GeoDataFrame
neighbourhoods.head(10)

## Question 5c
Neighbourhoods `Waterfront Communities-The Island`, `Bay Street Corridor`, `Church-Yonge Corridor`, and `Niagara` have the highest number of bike stationa. However, counting the number of stations in a certain geographic region is not the best way to assess access to the bike share network in a neighbourhood because the areas of each neighbourhood are different. Thus, it would be better to calculate a station density in units of `stations per square km`. Create a new column in `neighbourhoods` called `'station_density'` and assign each neighbourhood's station density to it.

In [ ]:
# Write your code here.
neighbourhoods['station_density'] = neighbourhoods['stations'] / neighbourhoods['area']

# View GeoDataFrame
neighbourhoods.head(10)

# 8. Interactive Maps
In this section, we'll create interactive maps with the folium package to visualize our neighbourhood and station data.

Let's first install folium.

In [ ]:
!pip install folium

Let's import folium.

In [ ]:
import folium

Let's start by plotting a simple interactive map of Toronto (43.6426, -79.3871) using `folium.Map()`.

**IMPORTANT:** follium expects geographic locations in (latitude, longitude). Therefore, if you're working in a `crs` that uses (East, North), like we have been (`EPSG:26917`), then you'll need to use `.to_crs()` to transform to another `crs`. We'll be using `.to_crs(epsg=4326)`. 

In [ ]:
# Create a map of Toronto
map_1 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Display map
map_1

There are some arguments for `folium.Map()` we need to unpack.

```python
folium.Map(location=[43.6426, -79.3871],
           tiles='cartodbpositron', 
           zoom_start=12)
```

- `location`: The initial center of the map. (43.6426, -79.3871) is the location of the CN Tower in downtown Toronto.
- `tiles`: Controls the styling of the map. We chose the `'cartodbpositron'` style. Try out the `'openstreetmap'` style. More styles [here](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).
- `zoom_start`: Sets the initial level of zoom of the map. Higher values of `zoom_start` zoom in closer to the map.

## Marker
Next, let's create a create a map using follium and place a `Marker()` at the location of each bike station.

In [ ]:
from folium import Marker

In [ ]:
# Create a map of Toronto
map_2 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    Marker([row.geometry.y, row.geometry.x]).add_to(map_2)

# Display map
map_2

## MarkerCluster
Next, let's create a create a map using follium and the `MarkerCluster` plugin. The map we just created above is a bit cluttered before we zoom to the neighbourhood scale. `MarkerCluster` will help declutter the map. 

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
# Create a map of Toronto
map_3 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
mc = MarkerCluster()
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    mc.add_child(Marker([row.geometry.y, row.geometry.x]).add_to(map_2))

map_3.add_child(mc)
    
# Display map
map_3

## Circle
Next, let's create a create a map using follium and the `Circle` object. The map we just created above is definitely less cluttered but we're limited in the amount of information we can display. By using the `Circle` object, we can vary the size and color of each `Circle` and therefore, show the relationship between location and two other variables. 

In [ ]:
from folium import Circle

In [ ]:
# Create a map of Toronto
map_4 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    Circle(location=[row.geometry.y, row.geometry.x],
           radius=20,
           color='forestgreen').add_to(map_4)

# Display map
map_4

There are some arguments for `Circle()` we need to unpack.

```python
Circle(location=[row.geometry.y, row.geometry.x],
       radius=20,
       color='forestgreen').add_to(map_4)
```

- `location`: The initial center of the circle in (latitude, longitude). 
- `radius`: The radius of each circle.
- `color`: The color of each circle.

## HeatMap
Next, let's create a create a map using follium and the `HeatMap` function. The `HeatMap` will show the density of bike stations in different areas of the city, where red areas have relatively more bike stations. As you'd expect for a big city like Toronto, most of the bike stations are located near the center.

In [ ]:
from folium.plugins import HeatMap

In [ ]:
# Create a map of Toronto
map_5 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
HeatMap(data=list(zip(bikeshare_stations_gdf.to_crs(epsg=4326).geometry.y,
                      bikeshare_stations_gdf.to_crs(epsg=4326).geometry.x)),
        radius=20).add_to(map_5)

# Display map
map_5

There are some arguments for `HeatMap()` we need to unpack.

```python
HeatMap(
    data=list(zip(bikeshare_stations_gdf.to_crs(epsg=4326).geometry.y,
                  bikeshare_stations_gdf.to_crs(epsg=4326).geometry.x)),
    radius=20
).add_to(map_5)
```

- `data`: A DataFrame or list containing the locations (latitude, longitude) that we'd like to plot.
- `radius`: Controls the smoothness of the heatmap where higher values make the heatmap look smoother.

## Choropleth
Choropleth maps are used to represent statistical data through various shading patterns or symbols on predetermined geographic areas (i.e. countries, provinces, neighbourhoods, etc.). To understand how access to the Toronto bike share network varies by neighbourhood, we'll create a choropleth map.

As a first step, we create a GeoDataFrame where each neughbourhood is assigned a different row, and the "geometry" column contains the geographical boundaries.

In [ ]:
plot_geography = neighbourhoods.to_crs(epsg=4326)[['name', 'geometry']]
plot_geography = plot_geography.set_index('name')
plot_geography.head()

Next, we'll create a DataFrame from `neighbourhoods` containing the quanity we want to plot `'station_density'` and the neighbourhood name, which should match the index of `plot_geography`.

In [ ]:
plot_data = neighbourhoods[['name', 'station_density']]
plot_data.head()

In [ ]:
from folium import Choropleth

In [ ]:
# Create a base map
map_6 = folium.Map(location=[43.6426, -79.3871], 
                 tiles='cartodbpositron',
                 zoom_start=10)

# Add a choropleth map to the base map
Choropleth(geo_data=plot_geography.__geo_interface__, 
           columns=['name', 'station_density'],
           data=plot_data, 
           key_on='feature.id', 
           fill_color='YlOrRd', 
           legend_name='Bikeshare Station Density (stations / km**2)'
          ).add_to(map_6)

# Display the map
map_6

There are some arguments for `Choropleth()` we need to unpack.

```python
Choropleth(geo_data=districts.__geo_interface__, 
           columns=['name', 'station_density'],
           data=plot_dict, 
           key_on='feature.id', 
           fill_color='YlOrRd', 
           legend_name='Bikeshare Station Density (stations / km**2)'
          ).add_to(map_6)
```

- `geo_data`: A GeoJSON FeatureCollection containing the boundaries of each geographical (neighbourhood area.
 - In the code above, we convert the districts GeoDataFrame to a GeoJSON FeatureCollection with the __geo_interface__ attribute. Try it out for yourself in a new cell to see what a GeoJSON looks like.
- `data`: A Pandas DataFrame containing the values that will be used to color-code each geographical area.
- `key_on`: Always be set to `'feature.id'`
- `fill_color`: Sets the colormap.

## Question 6
In Assignment 4, you merged weather station data with bike share ridership data. We chose a weather stations somewhere downtown but never looked at its location relative to the bike stations. 

Use folium to plot:
1. The station locations in `bikeshare_stations` using `Circle()` objects.
2. The location of the TORONTO CITY CENTRE weather station `(43.63, -79.4)` using a `Marker()` object.

Your plot should look something like this.
<br>
<img src="images/q6.png" alt="drawing" width="700"/>
<br> 

In [ ]:
# Write your code here.

# Create a map of Toronto
map_4 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in bikeshare_stations_gdf.to_crs(epsg=4326).iterrows():
    Circle(location=[row.geometry.y, row.geometry.x],
           radius=20,
           color='forestgreen').add_to(map_4)

# add a marker for TORONTO CITY CENTRE weather station
folium.Marker(location=[43.6426, -79.3871]).add_to(map_4)

# Display map
map_4



## 9. Geocoding
Geocoding converts addresses into geographic coordinates to be placed on a map. If you have ever Googled a geographic location like "CN Tower", then you have used a geocoder.

First, we need to install another library called `geopy`. Let's use `pip install` to do this.

In [ ]:
!pip install geopy

In [ ]:
from geopandas.tools import geocode

Let's use the names of Toronto's three universities as our search terms.

In [ ]:
universities = geocode(['University of Toronto', 
                        'Ryerson University', 
                        'York University'])

Now, let's plot them up using `Marker()`!

In [ ]:
# Create a map of Toronto
map_7 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)

# Add points to the map
for idx, row in universities.to_crs(epsg=4326).iterrows():
    Marker([row.geometry.y, row.geometry.x]).add_to(map_7)

# Display map
map_7

We won't spend any more time on geocoding but its good to know about.

# 10. Spatial Joins
GeoDataFrames can be combined using the same methods we learned in Week 2 (`.concat()`, `.merge()`). But, there is another type of merging operation called a [**spatial join**](https://geopandas.org/reference/geopandas.sjoin.html) `.sjoin()`. With a spatial join, we combine GeoDataFrames based on the spatial relationship between the objects in the `'geometry'` columns. 

## Question 7
In a previous section, we combined `neighbourhoods` and `bikeshare_stations_gdf` by counting the number of stations in each neighbourhood. Add a new column to `bikeshare_stations_gdf` called `'neighbourhood'`, which contains the neighbourhood that a particular station is located in.

In [ ]:
# Write your code here.
#bikeshare_stations_gdf = gpd.sjoin( bikeshare_stations_gdf, neighbourhoods)
#bikeshare_stations_gdf = bikeshare_stations_gdf.drop(columns = ['index_right', 'area', 'stations', 'station_density'])
#bikeshare_stations_gdf = bikeshare_stations_gdf.rename(columns = {'name': 'neighbourhoods'})

# View GeoDataFrame
bikeshare_stations_gdf.head()


# 11. Proximity Analysis
In this section, you'll learn how to select all points within some radius or boundary of a geometry feature.  

Our last task is to find all TTC subways stations that are not within 200 meter of a bike station. 

## Question 8a
Import the `'subway_stations.shp'` shape file as a GeoDataFrame and transform the `crs` to `EPSG:26917`. Create a new variable called `subway_stations` and save the GeoDataFrame to it.

In [ ]:
# Write your code here.
subway_stations = gpd.read_file('subway_stations.shp')

#convert crs to EPSG:26917
subway_stations = subway_stations.to_crs(epsg=26917)

# View GeoDataFrame
subway_stations.head()

## Question 8b
We want to see whether TTC subway stations are within 200 meters of a bike stations and the simplest way to do this is to create a buffer. The `'geometry'` column in a GeoDataFrame has a method called `.buffer()`, which takes a radius argument in the units of your `crs` (meters in the case of `EPSG:26917` 

Create a new variable called `bikeshare_stations_buffer` and set it equal to `bikeshare_stations_gdf` with a 200 meter buffer applied.

In [ ]:
# Write your code here.
bikeshare_stations_buffer = bikeshare_stations_gdf.geometry.buffer(200)

# View GeoDataFrame
bikeshare_stations_buffer.head()

`bikeshare_stations_buffer` should be a geopandas.geoseries.GeoSeries data type.

In [ ]:
type(bikeshare_stations_buffer)

Next, let's plot `bikeshare_stations_buffer` and see what it looks like.

In [ ]:
from folium import GeoJson

In [ ]:
# Create map with release incidents and monitoring stations
map_8 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)
    
# Plot each polygon on the map
GeoJson(bikeshare_stations_buffer.to_crs(epsg=4326)).add_to(map_8)

# Show the map
map_8

We can see that each bike station is now represented as a 200 meter radius circle centered on the station's location.

Now we want to test if a subway station is within 200 meters of a bike station. We could test each subway station against each bike station buffer with a `for` loop but this would take a while. A more efficient way is to first collapse all of the buffer `POLYGON` into a `MULTIPOLYGON` object. We do this with the `unary_union` attribute.

In [ ]:
bike_station_union = bikeshare_stations_buffer.geometry.unary_union

Let's quickly see what it looks like

In [ ]:
bike_station_union

`bike_station_union` is just a `MULTIPOLYGON` (data type: `shapely.geometry.multipolygon.MultiPolygon`) so let's convert it to a GeoDataFrame.

In [ ]:
bike_station_union = gpd.GeoDataFrame(geometry=[bike_station_union], crs='EPSG:26917')
bike_station_union.head(100)

## Question 8c
Next, we want to know whether or not each subway station is within 200 meters of a bike share station. 

Create a new column in `subway_stations` called `'bike_access'` and assign it a boolean value. `True` if the station is within 200 meters of a bike station and `False` if it is not.

Hint: You can use the `.contains()` method to check if one geometry overlaps with another. 

```python
GeoDataFrame.geometry.contains(geometry)
```

You can use the `.apply()` method to loop through each row of `subway_stations`.

<br>
<img src="images/q8c.png" alt="drawing" width="700"/>
<br> 

In [ ]:
# Write your code here.
subway_stations['bike_access'] = subway_stations.apply(lambda row: bike_station_union.geometry.contains(row.geometry), axis = 1)

# View GeoDataFrame
subway_stations.head(100)

Let's plot your answer to see if it makes sense.

In [ ]:
# Create map with release incidents and monitoring stations
map_9 = folium.Map(location=[43.6426, -79.3871], 
                   tiles='cartodbpositron', 
                   zoom_start=10)
    
# Plot each polygon on the map
GeoJson(bike_station_union.to_crs(epsg=4326)).add_to(map_9)

# Add points to the map
for idx, row in subway_stations.to_crs(epsg=4326).iterrows():
    if row['bike_access']:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='green'),
               popup=row['STATION']).add_to(map_9)
    else:
        Marker([row.geometry.y, row.geometry.x], 
               icon=folium.Icon(color='red'),
               popup=row['STATION']).add_to(map_9)

# Show the map
map_9

## Question 8d
What percentage of TTC subway stations are within 200 meters of a bike stations? Create a new variable called `bike_access` and assign your answer to it.

In [ ]:
# Write your code here.
# we know from earlier part that number of rows are 65 so total entries would be 65
bike_access = (subway_stations['bike_access'].sum()/65)*100

# Print answer
print('{} % of subways stations are within 200 meters of a bike station.'.format(bike_access))

**Congratulation, you're done Assignment 5. Review your answers and clean up that code before submitting on Quercus. `#cleancode`**